In [47]:
import pandas as pd
import re

import nltk
import collections
import string
import numpy as np
import sklearn
import gzip
import csv
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [4]:
covid_19_tweets = pd.read_csv('COVID-19_tweets.csv')

In [5]:
covid_19_tweets

,Date,Tweet,User,Location,Belief,Keywords_used
0,11/18/2020,BREAKING Athletic Director Patricia Nicol has ...,b'ethanmmcdowell',"b'Boston, MA'",.,keywords
1,11/18/2020,I need the audience to realize COVID19 was cir...,b'salvadorandolar',b'Stolen Tongva Land',.,keywords
2,11/18/2020,thus it breaksdown and splits a chastisement t...,b'EDHKNse8ps3kkfa',b'',.,keywords
3,11/18/2020,Cant believe we just canceled the entire video...,b'RyanPinks',"b'Philadelphia, PA'",.,keywords
4,11/18/2020,12 This year will be a tough holiday season We...,b'joinandrewdo',"b'Orange County, CA '",.,keywords
...,...,...,...,...,...,...
7930,11/13/2020,Covid19 is about to collapse our hospital syst...,b'BeamMeUpScotee',"b'Oregon, USA'",.,keywords
7931,11/13/2020,Covid19 was the worst thing to happen to me me...,b'Kam_Bam_',b'arkadoo',.,keywords
7932,11/13/2020,1 new COVID19 case in New Providence 24 total ...,b'MayorMorgan1',"b'New Providence, NJ'",.,keywords
7933,11/13/2020,What about blaming essential workers for the s...,b'AG_HighFly5',b'Southside.Iowa City.',.,keywords


In [29]:
def clean_tweet(tweet):
    #Step 1 - drop links
    regex = re.compile(r'https?://t.co/[a-zA-Z0-9]{10}')
    remove_links = re.split(regex,tweet)
    remove_links = ' '.join(remove_links)

    #Step 2 - Remove any non-ascii characters
    remove_non_ascii =  re.sub(r'[^\x00-\x7F]+',' ', remove_links).lower()

    #Step 3 - check for apostrophes
    remove_apostroph = remove_non_ascii.replace("'s","")
    remove_apostroph = remove_apostroph.replace("'","")
    remove_apostroph = remove_apostroph.split(' ')
    
    #Step 4 - Remove dashes
    remove_dashes = ' '.join(remove_apostroph).split('-')
    remove_dashes = ' '.join(remove_dashes)

    #Step 5 - Keep letters and digits only
    keep_letters =  re.sub(r'[^a-zA-Z0-9]',' ', remove_dashes)

    keep_letters = keep_letters.split(' ')

    keep_letters = ' '.join(keep_letters)

    #Step 6 - Tokenize 
    tokenize = nltk.word_tokenize(keep_letters)

    #Step 7 - Lemmatize    
    lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()
    lemmatize_text = map(lambda x: lemmatizer.lemmatize(x), tokenize)

    #Step 8 - Remove stop words
    stopwords = set(nltk.corpus.stopwords.words('english'))
    remove_stop = list(filter(lambda x: x not in stopwords, lemmatize_text))

    #Step 9 - Remove any empty strings from list
    processed_list = list(filter(lambda a: a != "", remove_stop))

    processed_string = " ".join(processed_list)

    return processed_string

In [36]:
# print(clean_tweet(covid_19_tweets['Tweet'][100]))

In [43]:
tweets_list = covid_19_tweets['Tweet'].tolist()
# clean_tweet(covid_19_tweets.iloc['Tweet'])
for i,tweet in enumerate(tweets_list):
    tweets_list[i] = clean_tweet(tweet)

In [44]:
def do_nothing(x):
    return x

def tweet_features(tweets_list):

    tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer= do_nothing,
    preprocessor=do_nothing
    )
    vector = tfidf.fit_transform(tweets_list)
    return vector

print(tweet_features(tweets_list))

  (0, 35)	0.03097492585122846
  (0, 31)	0.05871754825658438
  (0, 10)	0.029615952483205088
  (0, 32)	0.05582769213654632
  (0, 1)	0.07390962796097907
  (0, 2)	0.058680463793106145
  (0, 33)	0.03452190026253273
  (0, 16)	0.14232896228746486
  (0, 29)	0.16611665454107094
  (0, 23)	0.029384754446254206
  (0, 26)	0.0889152924993178
  (0, 25)	0.2463029055423408
  (0, 14)	0.10992410954657496
  (0, 13)	0.32948157194411093
  (0, 22)	0.2534192671305025
  (0, 18)	0.060206733704782925
  (0, 30)	0.27484490533485323
  (0, 0)	0.4643602919334879
  (0, 17)	0.0899221602659032
  (0, 24)	0.19263402344105626
  (0, 19)	0.32827975231595447
  (0, 21)	0.03522572770493068
  (0, 11)	0.2740840268481674
  (0, 15)	0.35554678318163685
  (0, 28)	0.16582383555061248
  :	:
  (7934, 34)	0.10981143606423806
  (7934, 35)	0.11946088596766924
  (7934, 31)	0.13587345526386954
  (7934, 10)	0.04568789529198837
  (7934, 32)	0.06459313829702942
  (7934, 1)	0.11401879933680888
  (7934, 2)	0.06789382057655691
  (7934, 33)	0.13314

In [49]:
numeric = tweet_features(tweets_list)
kmeans = KMeans(n_clusters=2, random_state=0).fit(numeric)
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [52]:
centers = kmeans.cluster_centers_
centers

array([[0.4856942 , 0.01675673, 0.04265231, 0.0153635 , 0.0076918 ,
        0.00887472, 0.00701808, 0.00533019, 0.00726711, 0.00426801,
        0.03755127, 0.25928795, 0.05067212, 0.16956478, 0.15388626,
        0.30733207, 0.05957682, 0.10786396, 0.08623763, 0.30168422,
        0.01803561, 0.05403234, 0.1511904 , 0.09778793, 0.24522514,
        0.24533317, 0.09494011, 0.00873379, 0.1913488 , 0.17828868,
        0.23590002, 0.09493749, 0.07830751, 0.05694514, 0.01513646,
        0.06822081, 0.01550335],
       [0.50641723, 0.03739493, 0.04038307, 0.02555261, 0.01371184,
        0.01315978, 0.01324844, 0.01095803, 0.0095516 , 0.00966215,
        0.03433083, 0.24791652, 0.05859919, 0.13635421, 0.16059335,
        0.40736182, 0.06230269, 0.07836896, 0.09284837, 0.20808612,
        0.02028359, 0.05785216, 0.16364148, 0.09489492, 0.18546298,
        0.21754325, 0.10125948, 0.00891784, 0.19598215, 0.19278952,
        0.21018852, 0.09453644, 0.07173045, 0.06182038, 0.01886508,
        0.08064

In [48]:
plt.scatter(numeric[:, ], s=50);

TypeError: scatter() missing 1 required positional argument: 'y'